# Week 17: Advanced NLP - Sentiment & Event-Driven Trading

## 🎯 Learning Objectives

By the end of this week, you will understand:
- **Text Preprocessing**: Cleaning financial text
- **Sentiment Analysis**: VADER, TextBlob, FinBERT
- **Event Detection**: News impact analysis
- **Trading Signals**: From text to alpha

---

## Why NLP in Finance?

Markets react to information:
- Earnings calls
- News headlines
- Social media
- SEC filings

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
print("✅ Libraries loaded!")
print("📚 Week 17: NLP for Finance")

---

## Part 1: Text Preprocessing

### Steps

1. **Cleaning**: Remove HTML, special chars
2. **Tokenization**: Split into words
3. **Normalization**: Lowercase, stemming
4. **Stop words**: Remove common words

In [ ]:
def preprocess_text(text):
    """Basic text preprocessing"""
    # Lowercase
    text = text.lower()
    # Remove special characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Remove extra whitespace
    text = ' '.join(text.split())
    return text

# Example headlines
headlines = [
    "Apple Reports Record Q4 Earnings, Stock Surges 5%",
    "Tesla Misses Delivery Targets, Shares Tumble",
    "Fed Signals Rate Hikes Amid Inflation Concerns",
    "Microsoft Cloud Revenue Beats Expectations",
    "Oil Prices Crash on Global Demand Worries"
]

print("Text Preprocessing Example")
print("="*50)
for h in headlines[:2]:
    print(f"Original: {h}")
    print(f"Cleaned:  {preprocess_text(h)}\n")

---

## Part 2: Sentiment Analysis

### Rule-Based: VADER

- Valence Aware Dictionary and sEntiment Reasoner
- Good for short social media text
- Returns positive, negative, neutral, compound scores

In [ ]:
try:
    from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
    
    analyzer = SentimentIntensityAnalyzer()
    
    print("VADER Sentiment Analysis")
    print("="*60)
    
    for headline in headlines:
        scores = analyzer.polarity_scores(headline)
        print(f"\n{headline}")
        print(f"  Compound: {scores['compound']:.3f}")
        sentiment = 'Positive' if scores['compound'] > 0.05 else ('Negative' if scores['compound'] < -0.05 else 'Neutral')
        print(f"  Label: {sentiment}")
        
except ImportError:
    print("⚠️ VADER not installed. pip install vaderSentiment")

---

## Part 3: Financial-Specific Sentiment

### Loughran-McDonald Dictionary

Standard sentiment dictionaries misinterpret financial language:
- "Liability" = negative in general, neutral in finance
- "Tax" = neutral in general, negative in finance

### FinBERT

BERT fine-tuned on financial text:
- Understands financial context
- State-of-the-art accuracy
- Handles complex sentences

In [ ]:
# Loughran-McDonald word lists (simplified)
lm_positive = ['profit', 'growth', 'success', 'gain', 'exceed', 'beat', 'strong', 'improve']
lm_negative = ['loss', 'decline', 'miss', 'fail', 'weak', 'concern', 'risk', 'crash']

def lm_sentiment(text):
    """Simple L-M sentiment score"""
    words = preprocess_text(text).split()
    pos = sum(1 for w in words if w in lm_positive)
    neg = sum(1 for w in words if w in lm_negative)
    total = pos + neg
    if total == 0:
        return 0
    return (pos - neg) / total

print("Loughran-McDonald Sentiment")
print("="*60)
for headline in headlines:
    score = lm_sentiment(headline)
    print(f"{headline[:50]:50} Score: {score:+.2f}")

---

## Part 4: Sentiment Trading Signal

### Strategy

1. Aggregate daily news sentiment
2. Create rolling sentiment score
3. Generate trading signal
4. Backtest

In [ ]:
# Simulate sentiment time series
n_days = 500
np.random.seed(42)

# Generate sentiment scores (would come from news/social media)
sentiment = np.random.randn(n_days) * 0.3 + np.sin(np.linspace(0, 10, n_days)) * 0.2

# Simulate stock returns with sentiment correlation
noise = np.random.randn(n_days) * 0.02
returns = 0.001 * sentiment + noise + 0.0001  # Small positive drift

# Create DataFrame
df = pd.DataFrame({
    'sentiment': sentiment,
    'returns': returns
})

# Rolling sentiment signal
df['sentiment_ma'] = df['sentiment'].rolling(5).mean()
df['signal'] = np.sign(df['sentiment_ma'])
df['strategy_returns'] = df['signal'].shift(1) * df['returns']

# Calculate performance
df = df.dropna()
cum_strat = (1 + df['strategy_returns']).cumprod()
cum_bh = (1 + df['returns']).cumprod()

print("Sentiment Strategy Performance")
print("="*50)
print(f"Strategy Total Return: {(cum_strat.iloc[-1] - 1):.2%}")
print(f"Buy & Hold Return: {(cum_bh.iloc[-1] - 1):.2%}")
print(f"Strategy Sharpe: {df['strategy_returns'].mean() / df['strategy_returns'].std() * np.sqrt(252):.2f}")

# Plot
fig, axes = plt.subplots(3, 1, figsize=(12, 8), sharex=True)

axes[0].plot(df['sentiment'], alpha=0.5, label='Daily')
axes[0].plot(df['sentiment_ma'], label='5-day MA')
axes[0].set_ylabel('Sentiment')
axes[0].legend()
axes[0].set_title('News Sentiment')

axes[1].plot(df['signal'])
axes[1].set_ylabel('Position')
axes[1].set_title('Trading Signal')

axes[2].plot(cum_strat, label='Sentiment Strategy')
axes[2].plot(cum_bh, label='Buy & Hold', alpha=0.7)
axes[2].set_ylabel('Cumulative Return')
axes[2].legend()
axes[2].set_title('Strategy Performance')

plt.tight_layout()
plt.show()

---

## Interview Questions

### Conceptual
1. Why do general sentiment dictionaries fail for finance?
2. How would you validate a sentiment model?
3. What are the latency considerations for news-based trading?

### Technical
1. How does FinBERT differ from regular BERT?
2. How would you handle multiple news sources?
3. What features would you extract from earnings calls?

### Finance-Specific
1. How do you avoid overfitting to historical news?
2. What's the typical decay rate of news sentiment alpha?
3. How would you combine sentiment with other signals?

---

## Key Takeaways

| Method | Speed | Accuracy | Use Case |
|--------|-------|----------|----------|
| VADER | Fast | Moderate | Social media |
| L-M | Fast | Financial | SEC filings |
| FinBERT | Slow | High | Complex analysis |